In [1]:
import pandas as pd
import numpy as np
from BokehStructureGraph import BokehStructureGraph as bsg
import datetime

In [2]:
confirmed_cases=pd.read_csv('csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
deaths = pd.read_csv('csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')

In [3]:
from bokeh.plotting import show, output_notebook, figure
from bokeh.models import ColumnDataSource, LogScale, DatetimeAxis, DatetimeTicker, DaysTicker
output_notebook()

Loading BokehJS ...

In [4]:
cases = confirmed_cases.groupby('Country/Region').sum().iloc[:,2:].transpose()

In [5]:
cases.index=pd.to_datetime(cases.index)
cases.index.name='Date'
from sklearn.linear_model import LinearRegression

L=LinearRegression().fit(np.array(range(len(cases['US']))).reshape(-1,1),np.log(cases['US']+1))

y=L.predict(np.array(range(len(cases['US']))).reshape(-1,1))

cases['predicted']=np.exp(y)

source = ColumnDataSource(cases)
source=ColumnDataSource(cases)

In [14]:
F = figure(x_axis_type='datetime',y_axis_type='log',y_range=(1,100000),x_range=(datetime.datetime(2020,1,1),datetime.datetime(2020,5,1)))


In [15]:
F.line(x='Date',y='US',source=source,line_width=3,color='darkblue')
F.line(x='Date',y='predicted',color='red',line_width=3,source=source)
show(F)

In [58]:
M=bsg.BokehStructureGraph(F)

In [59]:
show(M.model)

In [60]:
F.below[0].ticker.tickers=[F.select_one({'id':'18904'})]